# Decision Theory Project. AI System for Minesweeper

##### Bohdan Tymofieienko, B. S. 
###### btymofieienko@student.fontys.nl
##### Mikolaj Hilgert, M. A. 
###### m.hilgert@student.fontys.nl


##### 23 June 2022

### 1. Table of Contents <a class="anchor" id="chapter1"></a>


* [1. Table of Contents](#chapter1)
* [2. Revision history](#chapter2)
* [3. Abstract](#chapter3)
* [4. Introduction](#chapter4)
* [5. Environment](#chapter5)
* [6. Rewards structure](#chapter6)
* [7. Transitional probabilities and Stochasticity](#chapter7)
* [8. Markov Decision Process](#chapter8)
* [9. Random Agent](#chapter9)
* [10. Value iteration. Abstract Policy](#chapter10)
* [11. Rational Agent](#chapter11)
* [12. Reasoning with results. Optimizations](#chapter12)
* [13. Conclusion](#chapter13)
* [14. Appendix A. Reflection](#chapter14)
* [15. Appendix B. References](#chapter15)

### 2. Revision history <a class="anchor" id="chapter2"></a>

| Version | Author          | Date  | Change                                       |
|---------|-----------------|-------|----------------------------------------------|
| 0.1     | Bohdan, Mikolaj | 18.05 | Added notebook and game |
| 0.2  | Bohdan, Mikolaj | 11.06 | Added environment and rational agent. |
| 1.0   | Bohdan, Mikolaj | 23.06 | Added conclusions and reasoning. |


### 3. Abstract <a class="anchor" id="chapter3"></a>

For our decision theory project we decided to build an Artificial intelligence system in form of rational agent for minesweeper game. We used a value iteration approach that created an abstract policy with which our agent wins on average 3 to 4 times more games than a random agent.

### 4. Introduction  <a class="anchor" id="chapter4"></a>

We chose a minesweeper game because it is a classical game that everyone can recognise and it requires a certain cognitive process that is an interesting challenge to simulate. We as not not avid players, wanted to see how an agent would fare in a game by learning the abstract rules. Our end goal was to make a rational agent that plays on a small scale board and does better than a random agent. As a proof of concept we chose to limit a board side but the idea should expand to any N x M times board. We initially took in consideration that the approach may lead to computational problems, thus we found it optimal on 2 x 3 board.

### 5. Environment  <a class="anchor" id="chapter4"></a>

We built our environment entirely from scratch which was useful in hindsight as we had to have full control and understanding of the game logic, to adapt rules. One change we made to the original game to make it more "machine-friendly" is that we removed flags from the possible actions. In order to win an agent shall open all the non-bombs cells. The environment is customisable with the size of the board, the bomb amount as well as the used seed for bomb generation. This allows for full control of the game, as you may retain the same arrangement of bombs and cells across games by passing a fixed random-seed.

Our environment consists of two classes, game which is essentially just an interface for the environment to interact  with the game and environment class that encapsulates the game logic and exposes information to an agent. 

As we said before through the environment agent can do steps and receive all the necessary information. Environment can also render a board to demonstrate a state of the game. Environment is also in charge of transition logic and reward policy. 

* Action space: all the cells on a board
* States: board itself is a state. For example: 
   
 
        1 1 
        # #
        
        s = {((0,0), 1), ((0,1), 1)}
        
        # - unopened cells
        
We decided to model our state-action space in this way because it is a simple way to compress all the information needed per step. We chose to use a tuple because it's convinient to extract values in python.

In [1]:
import random
from random import randint, choice

In [2]:
class Game:
  def __init__(self, n, m, bombs, seed = None):
    self.__seed = seed
    self.bombsCount = bombs
    self.status = 0
    self.board = [["#" for x in range(n)] for y in range(m)]
    self.bombs = []
    if seed != None:
      self.__generateBombs()
    
      
  def __generateBombs(self, x = None, y = None):
    if self.__seed != None:
        random.seed(self.__seed)
    while(len(self.bombs)<self.bombsCount):
      r_y = random.randint(0, len(self.board[0])-1)
      r_x = random.randint(0, len(self.board)-1)
      if self.__seed == None:
        if r_x != x and r_y != y and (r_x,r_y) not in self.bombs:
          self.bombs.append((r_x,r_y))
      else:
        if (r_x,r_y) not in self.bombs:
          self.bombs.append((r_x,r_y))
      
  def checkWin(self):    
    if sum(row.count('#') for row in self.board) == self.bombsCount:
      self.status = 1
  
  def getUnopenedCells(self):      
    moves = []
    for i in range(0, len(self.board)):
      for j in range(0, len(self.board[0])):
        if self.board[i][j] == "#":
          moves.append((i,j))
    return moves
  
  def getBoardConfig(self, bombsList):
    self.bombs = bombsList
    for i in range(len(self.board)):
      for j in range(len(self.board[0])):
          self.openCell(i,j)
    return self.board
    

  def openCell(self, x, y):
    if len(self.bombs) == 0:
      self.__generateBombs(x,y)
    if (x,y) in self.bombs:
      self.status = -1
      self.board[x][y] = "B"
    else:
      #Logic of openning
      self.__recOpen(x,y)
      self.checkWin()

    
  def __recOpen(self, x, y):
    adj = [(a, b) for a, b in [(x - 1, y), (x, y - 1), (x + 1, y), (x, y + 1),(x - 1, y-1), (x+1, y +1),(x + 1, y-1), (x-1, y + 1)]]
    counter = 0
    safeCells = []
    for test in adj:
        if test[0]<len(self.board) and test[1]<len(self.board[0]) and test[0] >= 0 and test[1] >= 0:
          if test in self.bombs:
              counter += 1
          else:
              safeCells.append(test)
    if counter==0:
      self.board[x][y] = "_"
    else:
      self.board[x][y] = str(counter)
    if self.board[x][y] == "_":
      for cell in safeCells:   
        if self.board[cell[0]][cell[1]] == "#":
          self.__recOpen(cell[0],cell[1])

### 6.  Rewards structure  <a class="anchor" id="chapter6"></a>

1) *Winning state*, so the state with all the cells opened except bombs results in  the reward of 10

In order to overweight all the negative rewards the optimal positive reward is 10. This has to do with how the Q-Value is       calculated. Since the transitional probability is included in the formula it is important that the state with the positive       reward contrasts from the losing state.  

2) *Losing state*, so the state with the opened bomb results in the reward of -1

Relatively large negative reward is essential part of the reward structure since it leads the agent to not make decisions that result in negative rewards as the agents **core** purpose is to maximize attained reward.

3) All the other states (-0.04)

An agent receives a small negative reward for non end states (win or loss) in order to motivate an agent to solve the game in the least amount of steps.  


### 7.  Transitional probabilities and Stochasticity  <a class="anchor" id="chapter7"></a>

After an iterative design process we understood that we are dealing with the stochastic environment since an agent may face a list of possible states it can end up at and calculate their probabilities. Initially we designed out environment in a way that every transition would give your guaranteed desired state. Meaning there is no drift. 

However, trying to make an agent solve any minesweeper of the certain size we realized there is a need for an abstract policy. Abstract policy implies that it was not based on fixed bombs configuration but is rather suitable for any configuration. The fact of the configuration being not disclosed until the end of the game adds stochasticity. We compensate that with the transitional policy. Consider the example below:
            
            Current state:
            
            1 1
            # #

            s = {((0,0), 1), ((0,1), 1)}
            a = (1,0)

            Possibilities:
                s_1 = {((0,0), 1), ((0,1), 1), ((1,0), 1)}
                s_2 = {((0,0), 1), ((0,1), 1), ((1,0), B)}

            P(s_1 | s, a) = 1/2  (Reward: 10)
            P(s_2 | s, a) = 1/2  (Reward: -1)
            
In fact if an agent is at state s it can end up in two different states with the probability of $\frac{1}{2}$ each. This idea extends for more possible states. 

The general formula is:

$$\frac{1}{N (Derivatives(s,a))}$$

*Derivatives - possible states with the given action*


### 8.  Markov Decision Process   <a class="anchor" id="chapter8"></a>

According to the definition, markov decision processes (mdps) model decision making in discrete, stochastic, sequential environments. The essence of the model is that a decision maker, or agent, inhabits an environment, which changes state randomly in response to action choices made by the decision maker *[M.L. Littman, 2001]*. All of the factors described hold true for our environment. First of all our environment is discrete, since every time step is modeled as a distinct state (set of opened cells and hints). The environment is stochastic as described in previous section. Next step cannot be fully determined by an agent.

An environment satisfies the Markov property if its state signal compactly summarizes the past without degrading the ability to predict the future *[Mark Lee 2005]*. This is exactly the case for our environment, since every state is a compact representation of the board *(current state)*. However, the state doesn't contain any information related to past states. In other words, given an agent is in the pre-winning state *(one cell to be oppened to win the game)* it doesn't matter how the agent got to that state. All that matters is an optimal action an agent takes that gives the greatest chance to end in a winning state. 

In [67]:
from enum import Enum

class Environment:
  def __init__(self, n, m, bombs, reward_per_step, seed = None):
    self.__game = None
    if seed == None:
      self.__game = Game(n, m, bombs)
    else:
      self.__game = Game(n, m, bombs, seed)
    self.__actions = [(x,y) for x in range(m) for y in range(n)]
    self.__possibleStates = self.generateAllStates()
    self.__initial_state = []
    self.__state = self.__initial_state
    self.__reward_per_step = reward_per_step
    self.__states_id_map = dict(enumerate(self.__possibleStates))
    
  
  def getStatus(self):
    return self.__game.status
  
  def state_coords(self, state):
    temp = []
    for x in state:
      temp.append(state[0])
    return temp          
    
  def __getReward(self, state, action = None):
    count = 0
    for x in state:
      if x[1] == 'B':
        return -1
      count += 1
    if count == ((len(self.__game.board) * len(self.__game.board[0])) - self.__game.bombsCount):
      return 10
    return self.__reward_per_step
      
  
  def generateAllStates(self):
    temp = []
    m = len(self.__game.board)
    n = len(self.__game.board[0])
    for x in self.__actions:
      newgame = Game(n, m,self.__game.bombsCount)
      board = newgame.getBoardConfig([x])
      states = self.__generatePossibleStates([(x,y) for x in range(m) for y in range(n)],[x])
      for s in states:
        newstate = []
        for e in s:
          newstate.append((e, newgame.board[e[0]][e[1]]))
        temp.append(newstate)
    result = []
    for x in temp:
      if x not in result:
        result.append(x)
    return result
    
  def __combinations(self,iterable):
    if len(iterable) == 0:
        return [[]]
    combos = []
    for combo in self.__combinations(iterable[1:]):
      combos += [combo,combo + [iterable[0]]]
    return combos

  def __generatePossibleStates(self,iterable, bombs):
    temp = []
    for set_ in self.__combinations(iterable):
      if len(set(set_).intersection(set(bombs))) <= 1:
        temp.append(set_)
    return temp
  
  def __calculate_transition(self, action):  
    self.__state.append((action, self.__game.board[action[0]][action[1]]))
  
  def getCurrentState(self):
    return self.__state
  
  def removeAction(self,action):
    self.__actions.remove(action)
  
  def getIdByState(self, state):
    for key, value in self.__states_id_map.items():
      if set(value) == set(state):
        return key
      
  def getStateById(self, _id):
      return self.__states_id_map[_id]
  
  def isGameOver(self):
    if self.__game.status == 1 or self.__game.status == -1:
      return True
    else:
      return False
    
  def isDone(self, state):
    count = 0
    for x in state:
      if x[1] == 'B':
        return True
      count += 1
    if count == ((len(self.__game.board) * len(self.__game.board[0])) - 1):
      return True
    return False
 
  def reset(self):
    self.__game = Game(len(self.__game.board[0]), len(self.__game.board), self.__game.bombsCount)
    self.__state = self.__initial_state
    return self.__state
  
  def step(self, action):
    #Open cell
    self.__game.openCell(action[0],action[1])
    #Step logic
    self.__calculate_transition(action)
    observation = self.__state
    done = self.isDone(self.__state)
    reward = self.__getReward(self.__state)
    info = self.__game
    return observation, done, reward, info
  

  def render(self, powerMode=False):
    for line in self.__game.board:
      print(line)
    print("State: ",self.__state)
    print("Status: ",self.__game.status)
    print("Bombs: ",self.__game.bombsCount)
    if powerMode:  
      print("Bombs: ",self.__game.bombs)
  
  def get_possible_states(self):
    return self.__possibleStates
  
  def get_actions(self):
    return self.__actions
    
  def get_transition_prob(self, action, new_state, old_state=None):
    if self.isDone(old_state):
      return 0  
    
    #Conditions
    length = (len(new_state) == len(old_state) + 1)
    intersection = set(new_state).intersection(set(old_state)) == set(old_state)
    diff = list(set(new_state) - set(old_state))
    same_action = [item[0] for item in diff] == [action]
    
    if not (length and intersection and same_action):
      return 0
    
    count = 0
    for x in self.__possibleStates:
      length = (len(x) == len(old_state) + 1)
      intersection = set(x).intersection(set(old_state)) == set(old_state)
      diff = list(set(x) - set(old_state))
      same_action = [item[0] for item in diff] == [action]
      if length and intersection and same_action:
        count+=1
    
    return 1 / count
    
    
    
  def getReward(self, state, action = None):
    return self.__getReward(state, action)
    

### 9. Random Agent  <a class="anchor" id="chapter9"></a>

The policy function $\pi(s) \to a$ is the concrete implementation of the decision process of the agent (selection of an action $a$). In the cell below, you can see the effect of an agent with a random policy choosing an arbitrary action regardless of the new state.

In [5]:
import random
import time

env = Environment(3,3,2,0.04,1032)

def policy_random(env):
    # action is random choice from all actions in Action Space
    random.seed(None)
    cell = random.choice(env.get_actions())
    return cell
  
  
total_reward = 0.0
done = False
nr_steps = 0


while not done:
  next_cell = policy_random(env)
  state, done, reward, info = env.step(next_cell)
  total_reward += reward
  nr_steps += 1
  print('cell: {}, reward: {:5.2f}'
          .format(next_cell, reward))
  for row in info.board:
    print(*row, sep="\t")
  print(info.bombs)
print('Episode done after {} steps. total reward: {:6.2f}. Result: {}'.format(nr_steps, total_reward, info.status))

cell: (0, 1), reward:  0.04
#	1	#
#	#	#
#	#	#
[(2, 1), (1, 2)]
cell: (2, 1), reward: -1.00
#	1	#
#	#	#
#	B	#
[(2, 1), (1, 2)]
Episode done after 2 steps. total reward:  -0.96. Result: -1


In [100]:
from statistics import mean, stdev

def run_one_episode(policy):
    env = Environment(2,3,1,-0.04)

    total_reward = 0.0
    done = False
    while not done:
        next_action = policy(env)
        state, done, reward, info = env.step(next_action)
        total_reward += reward
    if env.getStatus() == 1:
      return 1
    return 0

def measure_performance(policy, nr_episodes=100):
    N = nr_episodes
    print('statistics over', N, 'episodes')
    all_rewards = []
    for _ in range(N):
        episode_reward = run_one_episode(policy)
        all_rewards.append(episode_reward)

    print('mean: {:6.2f}, sigma: {:6.2f}'.format(mean(all_rewards), stdev(all_rewards)))
    print()
    for n, episode_reward in enumerate(all_rewards[:5], 1):
        print('ep: {:2d}, total reward: {:5.2f}'.format(n, episode_reward))
    print('......')
    for n, episode_reward in enumerate(all_rewards[-5:], len(all_rewards)-5):
        print('ep: {:2d}, total reward: {:5.2f}'.format(n, episode_reward))

measure_performance(policy_random)  # in Python a function pointer is simply the name of the function

statistics over 100 episodes
mean:   0.13, sigma:   0.34

ep:  1, total reward:  0.00
ep:  2, total reward:  1.00
ep:  3, total reward:  0.00
ep:  4, total reward:  0.00
ep:  5, total reward:  0.00
......
ep: 95, total reward:  0.00
ep: 96, total reward:  0.00
ep: 97, total reward:  0.00
ep: 98, total reward:  0.00
ep: 99, total reward:  0.00


### 10. Value iteration. Abstract Policy. <a class="anchor" id="chapter10"></a>


We chose a value iteration approach because of the clear structure and nature of the problem. 

In essence our end-goal is an abstract policy. We can think of it as of definitive reference for an agent with an instruction on what to do at each state. In order to extract such a policy we used a value iteration procedure. It allows us to numerically calculate the values of the states of Markov decision processes, with known transition probabilities and rewards. 


With the defined so-called Q-function we can calculate that numerical value 

(1) $Q(s, a) = \sum_{s'} P(s' \mid s, a) [R(s, a, s') + \gamma \space U(s')]$

The solution of the problem reduces to solving Bellman's equation

(2) $U_{i+1}(s) = \underset{a}{max} \sum_{s'} P(s' \mid s, a) \space [ R(s, a, s') + \gamma \space U_i(s') ]$

which can be simplified using the the Q-function definition as following

(3) $U_{i+1}(s) = \underset{a}{max} \space Q_i(s, a)$

It is also proved that Value Iteration is guaranteed to converge to the optimal values.

In [ ]:
def get_initial_U(mdp):
    U = {}
    for s in mdp.get_possible_states():
        U[mdp.getIdByState(s)] = mdp.getReward(s)
    return U
    
def Q_Value(mdp, s, a, U):
    Q = 0.0
    possible_states = mdp.get_possible_states()
    for s_p in possible_states:
        P = mdp.get_transition_prob(a, s_p, s)
        R = mdp.getReward(s_p, a)
        Q += P * (R + U[mdp.getIdByState(s_p)])
    return Q

def ValueIteration(mdp, error=0.00001):
    # from AIMA 4th edition without discount gamma 
    U_p = get_initial_U(mdp) # U_p = U'
    delta = float('inf')
    while delta > error:
        U = {}
        for s in mdp.get_possible_states():
            U[mdp.getIdByState(s)] = U_p[mdp.getIdByState(s)]
        print_U(U)  # to illustrate the iteration process
        delta = 0
        for s in mdp.get_possible_states():
            max_a = float('-inf')
            for a in mdp.get_actions():
                q = Q_Value(mdp, s, a, U) 
                if q > max_a:
                    max_a = q
            U_p[mdp.getIdByState(s)] = max_a
            if abs(U_p[mdp.getIdByState(s)] - U[mdp.getIdByState(s)]) > delta:
                delta = abs(U_p[mdp.getIdByState(s)] - U[mdp.getIdByState(s)])
    return U

def print_U(U):
    print('Utilities:')
    for key, value in U.items():
      print(mdp.getStateById(key), '->', value) 
    print('                   ', end = '')

def print_policy(pi):
    print('Policy:')
    for key, value in pi.items():
      print(mdp.getStateById(key), '->', value)
    print('                   ', end = '')


mdp = Environment(2,3,1,-0.04,1)
U = ValueIteration(mdp)
print_U(U)

pi_star = {}
for s in mdp.get_possible_states():
    if mdp.isDone(s):
        continue # policy is not needed in stop states
    max_a = float('-inf')
    argmax_a = None
    for action in mdp.get_actions():
        q = Q_Value(mdp, s, action, U) 
        if q > max_a:
            max_a = q
            argmax_a = action
    pi_star[mdp.getIdByState(s)] = argmax_a

print_policy(pi_star)

### 11. Rational agent  <a class="anchor" id="chapter11"></a>

Our rational agent works as following. By the rules of the game, one cannot fail at the first turn. First turn an agent picks a random cell. After that the agent looks up the current state in the reference (abstract policy) and does the action. This process is repeated until the game ends.

In [123]:
def RationalAgentPlay(env,pi_star, render = False):
  random.seed(None)
  cell = random.choice(env.get_actions())
  env.step(cell)
  if render: env.render(True) 
  while not env.isGameOver():
    state = env.getCurrentState()
    next_move = pi_star[env.getIdByState(state)]
    env.step(next_move) 
    if render: env.render(True) 
  return env.getStatus()

We gathered some statistics on our rational agent behavior. We can see that our rational agent wins with the probability of approximately 0.57 comparing to 0.13 for a random agent.

In [135]:
from statistics import mean, stdev

def run_one_episode(policy):
    env = Environment(2,3,1,-0.04)
    if(RationalAgentPlay(env,pi_star) == 1):
      return 1
    return 0

def measure_performance(policy, nr_episodes=100):
    N = nr_episodes
    print('statistics over', N, 'episodes')
    all_rewards = []
    for _ in range(N):
        episode_reward = run_one_episode(policy)
        all_rewards.append(episode_reward)

    print('mean: {:6.2f}, sigma: {:6.2f}'.format(mean(all_rewards), stdev(all_rewards)))
    for n, episode_reward in enumerate(all_rewards[:5], 1):
        print('ep: {:2d}, game result: {:5.2f}'.format(n, episode_reward))
    print('......')
    for n, episode_reward in enumerate(all_rewards[-5:], len(all_rewards)-5):
        print('ep: {:2d}, game result: {:5.2f}'.format(n, episode_reward))

measure_performance(policy_random)  # in Python a function pointer is simply the name of the function

statistics over 100 episodes
mean:   0.57, sigma:   0.50
ep:  1, game result:  0.00
ep:  2, game result:  1.00
ep:  3, game result:  1.00
ep:  4, game result:  0.00
ep:  5, game result:  0.00
......
ep: 95, game result:  0.00
ep: 96, game result:  0.00
ep: 97, game result:  1.00
ep: 98, game result:  0.00
ep: 99, game result:  0.00


### 12. Reasoning with results. Optimizations. <a class="anchor" id="chapter12"></a>

With the given attained results, we can see that our rational agent fares much better than pure random choice. The approach using value iteration was a suitable choice as we were able to create an abstract policy which can be adapted to any bomb and/or board size configuration. As mentioned before however, value iteration is computationally a very inefficient and expensive algorithm, due to the shear amount of possible states, especially with board of larger sizes. As that results in an exponentially larger size of potential states. This can of course be solved with adding hardware or with smarter programming. 

Another way to improve the performance could be to make use of parallel computing for value iteration. value iteration algorithms iterates trough all the possible actions for the state which could be parallelized and would lead to better performance.  

If we had to do a similiar project we would direct our research in the field of Deep Reinforcement Learning. Based on our experience we can see that there is a need in more sophisticated algorithm. To get better results we would also take discount factor in consideration, since as an agent approaches the end off the game it should be more careful at actions it takes. The idea would be to train a neural network to approximate a Q function instead of calculating it with all the possible states. This would give a space for much larger boards. 

The results of the two approaches (Random vs Rational) agents can be seen below:

![alt text](https://i.imgur.com/HKcJLNL.png)

### 13. Conclusion <a class="anchor" id="chapter13"></a>

Choice of the minesweeper game gave us a lot of potential for the design of the artificial intelligence system. By designing an abstract rules, in particular by defining a reward structure and transitional probabilities we built an agent that is capable of playing the game approximately 3-4 times better than a random agent. Although the results can be improved we think that this is a significant difference to describe the behavior of our agent as intelligent. 

### Appendix A. Reflections  <a class="anchor" id="chapter14"></a>

To start with, in hindsight a problem for our first reinforcement learning project was quite complicated. As it deviated quite a bit from the examples that were available. We had a hard time understanding the correct approach to such a problem. Not only it required a lot of research but also a new mindset, since we have never had an experience of working with stochastic problems. The literature we found during the research was often written in a very complex manner that we cannot yet understand. However, we are quite happy with the results we obtained and the knowledge we acquired during this project. Clearly this gave us a bigger picture of the artificial intellingence domain. It's hard to underestimate the intensity of computations needed for AI systems. As a result we had to downscale and simplify our game. Computational problems is another interesting discovery for us as software engineering students. 


### Appendix B. References <a class="anchor" id="chapter15"></a>

1. Markov decision process Markov Decision Process - an overview | ScienceDirect Topics, https://www.sciencedirect.com/topics/computer-science/markov-decision-process#:~:text=Markov%20decision%20processes%20mdps,made%20by%20the%20decision%20maker. Franciszek Grabski, 2001.


2. Reinforcement Learning 3.10 Summary, http://www.incompleteideas.net/book/ebook/node37.html . Mark Lee, 2005.
